In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats, interpolate
import scipy
import warnings
import datetime as dt
import landau

import importlib

# local imports
from lib import glob
from lib import plot
from lib import branches
from lib.constants import *

from scipy.integrate import quad, tplquad
from scipy.special import erf
from scipy.interpolate import CubicSpline


import ROOT

In [ ]:
importlib.reload(glob)
importlib.reload(plot)
importlib.reload(branches)

In [ ]:
dosave = True
savedir = "/icarus/app/users/gputnam/calib/plots_Feb15/"
plt.rcParams.update({'font.size': 14})
plotqual = "_mc"
plottitle = "MC"
tpcnames = ["EE", "EW", "WE", "WW"]

In [ ]:
#data = ntuples.dataframe(nproc="auto")
# perhit_df = pd.read_hdf('/icarus/data/users/gputnam/calib/mc/MCP2021G2/etau_C1.hdf5')
# perhit_df = pd.read_hdf('/icarus/data/users/gputnam/MCP2021P-data-calib/etau2.hdf5')

perhit_df = pd.read_hdf('/icarus/data/users/gputnam/MCP2021P-calib/etau2.hdf5')
# perhit_df = pd.read_hdf('/icarus/data/users/gputnam/MCP2021P-calib/etau2_nodiff.hdf5')
# perhit_df = pd.read_hdf('/icarus/data/users/gputnam/MCP2021P-calib/etau2_noscediff.hdf5')

In [ ]:
perhit_df.ccross_t0 - perhit_df.pandora_t0

In [ ]:
perhit_df["chunk10"] = perhit_df.index.get_level_values(1) // 10
perhit_df["charge"] = perhit_df.dqdx * perhit_df.pitch
perhit_df["thit"] = (perhit_df.time * tick_period - perhit_df.pandora_t0 - tanode*tick_period) / 1000.
perhit_df["tpcW"] = ~perhit_df["tpcE"]


perhit_df["tpcEE"] = perhit_df.tpcE & (perhit_df.cryostat == 0)
perhit_df["tpcEW"] = perhit_df.tpcW & (perhit_df.cryostat == 0)
perhit_df["tpcWE"] = perhit_df.tpcE & (perhit_df.cryostat == 1)
perhit_df["tpcWW"] = perhit_df.tpcW & (perhit_df.cryostat == 1)

In [ ]:
def landau_gaus(X, *p):
    mpv, eta, sigma, A = p
    sigma = np.minimum(sigma, 100*eta)
    return landau.landau.gauss_landau(X, mpv, eta, sigma, A)


In [ ]:
pitch_10 = perhit_df.groupby(["entry", "chunk10"]).pitch.sum()
dqdx_10 = perhit_df.groupby(["entry", "chunk10"]).charge.sum() / pitch_10
thit_10 = perhit_df.groupby(["entry", "chunk10"]).thit.mean()

tpc_EE_10 = perhit_df.groupby(["entry", "chunk10"]).tpcEE.all()
tpc_EW_10 = perhit_df.groupby(["entry", "chunk10"]).tpcEW.all()
tpc_WE_10 = perhit_df.groupby(["entry", "chunk10"]).tpcWE.all()
tpc_WW_10 = perhit_df.groupby(["entry", "chunk10"]).tpcWW.all()

In [ ]:
var = perhit_df.pandora_t0 - perhit_df.ccross_t0
when = (perhit_df.pitch < 0.4)

_ = plt.hist(var[when], bins=np.linspace(-1e4, 1e4, 21))

In [ ]:
_ = plt.hist(perhit_df.thit[(perhit_df.pitch < 0.4)], bins=np.linspace(0, 1000, 21))

In [ ]:
perhit_df["true_pitch"] = perhit_df.true_nelec / perhit_df.true_dqdx
perhit_df["true_dedx"] = perhit_df.true_e / perhit_df.true_pitch

In [ ]:
true_pitch_10 = perhit_df.groupby(["entry", "chunk10"]).true_pitch.sum()
true_dedx_10 = perhit_df.groupby(["entry", "chunk10"]).true_e.sum() / true_pitch_10
true_dqdx_10 = perhit_df.groupby(["entry", "chunk10"]).true_nelec.sum() / true_pitch_10

In [ ]:
tbins = np.linspace(0, 1000, 21)
tbins = tbins[:-1]
tcenters = (tbins[1:] + tbins[:-1])/2.

qbins = np.linspace(100, 4000, 39*4+1)
qbin_centers = (qbins[1:] + qbins[:-1]) / 2.

when = (perhit_df.pitch < 0.4)
tpcs = [perhit_df.tpcEE, perhit_df.tpcEW, perhit_df.tpcWE, perhit_df.tpcWW]

In [ ]:
Means = {}
MPVs = {}
MPV_errs = {}
locs = {}
loc_errs = {}

etas = {}
sigmas = {}

for tpc in tpcnames:
    Means[tpc] = []
    MPVs[tpc] = []
    MPV_errs[tpc] = []
    locs[tpc] = []
    loc_errs[tpc] = []
    etas[tpc] = []
    sigmas[tpc] = []
    

iplt = 0
for tpc, tname in zip(tpcs, tpcnames):
    for i, (blo, bhi) in enumerate(zip(tbins[:-1], tbins[1:])):
        plt.figure(iplt)
        thiswhen = tpc & when & (perhit_df.thit > blo) & (perhit_df.thit < bhi)

        N,_,_ = plt.hist(perhit_df.dqdx[thiswhen], bins=qbins)

        maxbin = np.argmax(N)
        when_fit = np.abs(np.array(range(len(qbin_centers))) - maxbin) < 15 # within 15 bins

        mpv_lo = 500
        mpv_hi = 1000
        mpv0 = 750
        p0 = [mpv0, 50, 50, np.max(N)*400.]
        bounds = ([mpv_lo, 10, 10, np.max(N)*400./100.], [mpv_hi, 250, 250, np.max(N)*400.*100.])
        popt, perr = curve_fit(landau_gaus, qbin_centers[when_fit], N[when_fit], 
                               p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N),1)[when_fit], bounds=bounds)

        plt.plot(qbin_centers, landau_gaus(qbin_centers, *popt), 
                 label="Fit")
        mpv = popt[0] - 0.228*popt[1]

        plt.text(0.45, 0.175, "1 Wire\n%.0f < time < %.0f $\\mu$s\n\nMPV = %.2f ADDC/cm\n$\\mu$ = %.2f ADDC/cm\n\n$\eta$ = %.2f ADDC/cm\n$\sigma$ = %.2f ADDC/cm" %
                 tuple([blo, bhi, mpv] + list(popt[:3])),
                 fontsize=14, transform=plt.gca().transAxes)
        plt.xlim([0, 3000])
        plt.xlabel("dQ/dx [ADDC/cm]")
        plt.ylabel("Entries")
        plt.title(plottitle + " TPC " + tname)
        plt.tight_layout()

        if dosave:
            plt.savefig(savedir + "1wire_t_%i_%i_dqdx_TPC%s%s.png" % (int(blo), int(bhi), tname, plotqual))

        Means[tname].append(np.mean(stats.trim1(perhit_df.dqdx[thiswhen], 0.2)))
        MPVs[tname].append(popt[0] - 0.228*popt[1])
        MPV_errs[tname].append(np.sqrt(np.diag(perr)[0] + (0.228**2)*np.diag(perr)[1] - 0.228*(perr[0,1] + perr[1,0])))

        locs[tname].append(popt[0])
        loc_errs[tname].append(np.sqrt(np.diag(perr))[0])

        etas[tname].append(popt[1])
        sigmas[tname].append(popt[2])
        
        iplt += 1
    
for tpc in tpcnames:
    Means[tpc] = np.array(Means[tpc])
    MPVs[tpc] = np.array(MPVs[tpc])
    MPV_errs[tpc] = np.array(MPV_errs[tpc])
    locs[tpc] = np.array(locs[tpc])
    loc_errs[tpc] = np.array(loc_errs[tpc])
    etas[tpc] = np.array(etas[tpc])
    sigmas[tpc] = np.array(sigmas[tpc])

In [ ]:
when10 = (pitch_10 < 4.)

tpc_10s = [tpc_EE_10, tpc_EW_10, tpc_WE_10, tpc_WW_10]

In [ ]:
Mean_10s = {}
MPV_10s = {}
MPV_err_10s = {}
loc_10s = {}
loc_err_10s = {}

eta_10s = {}
eta_10_errs = {}
sigma_10s = {}

ifig = 0

for tpc in tpcnames:
    Mean_10s[tpc] = []
    MPV_10s[tpc] = []
    MPV_err_10s[tpc] = []
    loc_10s[tpc] = []
    loc_err_10s[tpc] = []
    eta_10s[tpc] = []
    eta_10_errs[tpc] = []
    sigma_10s[tpc] = []

for tpc, tname in zip(tpc_10s, tpcnames):
    for i, (blo, bhi) in enumerate(zip(tbins[:-1], tbins[1:])):
        plt.figure(ifig)
        thiswhen = tpc & when10 & (thit_10 > blo) & (thit_10 < bhi)

        N,_,_ = plt.hist(dqdx_10[thiswhen], bins=qbins)

        maxbin = np.argmax(N)
        when_fit = np.abs(np.array(range(len(qbin_centers))) - maxbin) < 15 # within 15 bins

        mpv_lo = 500
        mpv_hi = 1000
        mpv0 = 750
        p0 = [mpv0, 50, 50, np.max(N)*400.]
        bounds = ([mpv_lo, 10, 10, np.max(N)*400./100.], [mpv_hi, 250, 250, np.max(N)*400.*100.])
        popt, perr = curve_fit(landau_gaus, qbin_centers[when_fit], N[when_fit], 
                               p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N),1)[when_fit], bounds=bounds)

        plt.plot(qbin_centers, landau_gaus(qbin_centers, *popt), 
                 label="Fit")
        mpv = popt[0] - 0.228*popt[1]

        plt.text(0.45, 0.175, "10 wires\n%.0f < time < %.0f $\\mu$s\n\nMPV = %.2f ADDC/cm\n$\\mu$ = %.2f ADDC/cm\n\n$\eta$ = %.2f ADDC/cm\n$\sigma$ = %.2f ADDC/cm" %
                 tuple([blo, bhi, mpv] + list(popt[:3])),
                 fontsize=14, transform=plt.gca().transAxes)
        plt.xlim([0, 3000])
        plt.xlabel("dQ/dx [ADDC/cm]")
        plt.ylabel("Entries")
        plt.title(plottitle + " TPC " + tname)
        plt.tight_layout()

        if dosave:
            plt.savefig(savedir + "10wires_t_%i_%i_dqdx_TPC%s%s.png" % (int(blo), int(bhi), tname, plotqual))

        Mean_10s[tname].append(np.mean(stats.trim1(dqdx_10[thiswhen], 0.2)))
        MPV_10s[tname].append(popt[0] - 0.228*popt[1])
        MPV_err_10s[tname].append(np.sqrt(np.diag(perr)[0] + (0.228**2)*np.diag(perr)[1] - 0.228*(perr[0,1] + perr[1,0])))

        loc_10s[tname].append(popt[0])
        loc_err_10s[tname].append(np.sqrt(np.diag(perr))[0])

        eta_10s[tname].append(popt[1])
        eta_10_errs[tname].append(np.sqrt(np.diag(perr))[1])
        sigma_10s[tname].append(popt[2])
        
        ifig += 1
    
for tpc in tpcnames:
    Mean_10s[tpc] = np.array(Mean_10s[tpc])
    MPV_10s[tpc] = np.array(MPV_10s[tpc])
    MPV_err_10s[tpc] = np.array(MPV_err_10s[tpc])
    loc_10s[tpc] = np.array(loc_10s[tpc])
    loc_err_10s[tpc] = np.array(loc_err_10s[tpc])
    eta_10s[tpc] = np.array(eta_10s[tpc])
    sigma_10s[tpc] = np.array(sigma_10s[tpc])

In [ ]:
whent = (tcenters > 100) & (tcenters < 900)

In [ ]:
def lifetime(t, Q0, tau):
    return Q0*np.exp(-t/tau)

p0 = [500, 3e3]
bounds = ([0.0, 0.0], [1500., 10e3])

In [ ]:
for i,tpc in enumerate(tpcnames):
    plt.figure(i)
    plt.errorbar(tcenters[whent], MPVs[tpc][whent], yerr=MPV_errs[tpc][whent], label="Data")
    
    popt, perr = curve_fit(lifetime, tcenters[whent], MPVs[tpc][whent], 
                       p0=p0, maxfev=10_000, sigma=MPV_errs[tpc][whent], bounds=bounds)
    plt.plot(tcenters[whent], lifetime(tcenters[whent], *popt), label="Fit")
    plt.text(0.05, 0.05, "$\\tau = %.0f \\pm %.0f \\mu$s" % (popt[1], np.sqrt(np.diag(perr))[1]),
            fontsize=16, transform=plt.gca().transAxes)
    
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("MPV dE/dx [MeV/cm]")
    plt.title(plottitle + " TPC " + tpc + " 1-Wire")
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "1wire_MPVdqdx_time_TPC%s%s.png" % (tpc, plotqual))

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    plt.plot(tcenters[whent], Means[tpc][whent])
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("Mean dQ/dx [ADDC/cm]")
    plt.title(plottitle + " TPC " + tpc + " 1-Wire")
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "1wire_Meandqdx_time_TPC%s%s.png" % (tpc, plotqual))

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    plt.plot(tcenters[whent], sigmas[tpc][whent])
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("Fit $\\sigma$ [ADDC/cm]")
    plt.title(plottitle + " TPC " + tpc + " 1-Wire")
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "1wire_sigma_time_TPC%s%s.png" % (tpc, plotqual))

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    plt.plot(tcenters[whent], etas[tpc][whent])
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("Fit $\\eta$ [ADDC/cm]")
    plt.title(plottitle + " TPC " + tpc + " 1-Wire")
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "1wire_eta_time_TPC%s%s.png" % (tpc, plotqual))

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    plt.errorbar(tcenters[whent], MPV_10s[tpc][whent], yerr=MPV_err_10s[tpc][whent])
    
    popt, perr = curve_fit(lifetime, tcenters[whent], MPV_10s[tpc][whent], 
                       p0=p0, maxfev=10_000, sigma=MPV_err_10s[tpc][whent], bounds=bounds)
    plt.plot(tcenters[whent], lifetime(tcenters[whent], *popt), label="Fit")
    plt.text(0.05, 0.05, "$\\tau = %.0f \\pm %.0f \\mu$s" % (popt[1], np.sqrt(np.diag(perr))[1]),
            fontsize=16, transform=plt.gca().transAxes)
    
    plt.title("10 Wires")
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("MPV dQ/dx [ADDC/cm]")
    plt.title(plottitle + " TPC " + tpc + " 10-Wire")
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "10wires_MPVdqdx_time_TPC%s%s.png" % (tpc, plotqual))

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    
    plt.plot(tcenters[whent], Mean_10s[tpc][whent])
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("Mean dQ/dx [ADDC/cm]")
    plt.title(plottitle + " TPC " + tpc + " 10-Wire")
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "10wires_Meandqdx_time_TPC%s%s.png" % (tpc, plotqual))

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    plt.plot(tcenters[whent], eta_10s[tpc][whent])
    plt.title("10 Wires")
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("Fit $\\eta$ [ADDC/cm]")
    plt.title(plottitle + " TPC " + tpc + " 10-Wire")
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "10wires_eta_time_TPC%s%s.png" % (tpc, plotqual))

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    
    plt.plot(tcenters[whent], sigma_10s[tpc][whent])
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("Fit $\\sigma$ [ADDC/cm]")
    plt.title(plottitle + " TPC " + tpc + " 10-Wire")

    plt.tight_layout()
    if dosave: plt.savefig(savedir + "10wires_sigma_time_TPC%s%s.png" % (tpc, plotqual))

In [ ]:
ebins = np.linspace(0.1, 10, 100)
ebin_centers = (ebins[1:] + ebins[:-1]) / 2.

In [ ]:
Means = {}
MPVs = {}
MPV_errs = {}
locs = {}
loc_errs = {}

etas = {}
sigmas = {}

for tpc in tpcnames:
    Means[tpc] = []
    MPVs[tpc] = []
    MPV_errs[tpc] = []
    locs[tpc] = []
    loc_errs[tpc] = []
    etas[tpc] = []
    sigmas[tpc] = []
    
ifig = 0

for tpc, tname in zip(tpcs, tpcnames):
    for i, (blo, bhi) in enumerate(zip(tbins[:-1], tbins[1:])):
        plt.figure(ifig)
        thiswhen = tpc & when & (perhit_df.thit > blo) & (perhit_df.thit < bhi)

        N,_,_ = plt.hist(perhit_df.true_dedx[thiswhen], bins=ebins)

        maxbin = np.argmax(N)
        when_fit = np.abs(np.array(range(len(ebin_centers))) - maxbin) < 15 # within 15 bins

        mpv_lo = 1
        mpv_hi = 3
        mpv0 = 2.
        p0 = [mpv0, 0.1, 0.1, np.max(N)]
        bounds = ([mpv_lo, 0.001, 0.001, np.max(N)/100.], [mpv_hi, 5, 5, np.max(N)*100.])
        popt, perr = curve_fit(landau_gaus, ebin_centers[when_fit], N[when_fit], 
                               p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N),1)[when_fit], bounds=bounds)

        plt.plot(ebin_centers, landau_gaus(ebin_centers, *popt), 
                 label="Fit")
        mpv = popt[0] - 0.228*popt[1]

        plt.text(0.45, 0.175, "1 Wire\n%.0f < time < %.0f $\\mu$s\n\nMPV = %.2f MeV/cm\n$\\mu$ = %.2f MeV/cm\n\n$\eta$ = %.3f MeV/cm\n$\sigma$ = %.3f MeV/cm" %
                 tuple([blo, bhi, mpv] + list(popt[:3])),
                 fontsize=14, transform=plt.gca().transAxes)
        plt.xlabel("dE/dx [MeV/cm]")
        plt.ylabel("Entries")
        plt.title(plottitle + " TPC " + tname)

        plt.tight_layout()

        if dosave:
            plt.savefig(savedir + "1wire_t_%i_%i_dedx_TPC%s%s.png" % (int(blo), int(bhi), tname, plotqual))

        Means[tname].append(np.mean(stats.trim1(perhit_df.dqdx[thiswhen], 0.2)))
        MPVs[tname].append(popt[0] - 0.228*popt[1])
        MPV_errs[tname].append(np.sqrt(np.diag(perr)[0] + (0.228**2)*np.diag(perr)[1] - 0.228*(perr[0,1] + perr[1,0])))

        locs[tname].append(popt[0])
        loc_errs[tname].append(np.sqrt(np.diag(perr))[0])

        etas[tname].append(popt[1])
        sigmas[tname].append(popt[2])
        
        ifig += 1
    
for tpc in tpcnames:
    Means[tpc] = np.array(Means[tpc])
    MPVs[tpc] = np.array(MPVs[tpc])
    MPV_errs[tpc] = np.array(MPV_errs[tpc])
    locs[tpc] = np.array(locs[tpc])
    loc_errs[tpc] = np.array(loc_errs[tpc])
    etas[tpc] = np.array(etas[tpc])
    sigmas[tpc] = np.array(sigmas[tpc])

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    
    plt.errorbar(tcenters[whent], MPVs[tpc][whent], MPV_errs[tpc][whent])
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("True MPV dE/dx [MeV/cm]")
    plt.title(plottitle + " TPC " + tpc + " 1-Wire")
    plt.tight_layout()

    if dosave: plt.savefig(savedir + "1wire_truededx_mpv_TPC%s%s.png" % (tpc, plotqual))

In [ ]:
Mean_10s = {}
MPV_10s = {}
MPV_err_10s = {}
loc_10s = {}
loc_err_10s = {}

eta_10s = {}
eta_10_errs = {}
sigma_10s = {}

for tpc in tpcnames:
    Mean_10s[tpc] = []
    MPV_10s[tpc] = []
    MPV_err_10s[tpc] = []
    loc_10s[tpc] = []
    loc_err_10s[tpc] = []
    eta_10s[tpc] = []
    eta_10_errs[tpc] = []
    sigma_10s[tpc] = []


ifig = 0

for tpc, tname in zip(tpc_10s, tpcnames):
    for i, (blo, bhi) in enumerate(zip(tbins[:-1], tbins[1:])):
        plt.figure(ifig)
        thiswhen = tpc & when10 & (thit_10 > blo) & (thit_10 < bhi)

        N,_,_ = plt.hist(true_dedx_10[thiswhen], bins=ebins)

        maxbin = np.argmax(N)
        when_fit = np.abs(np.array(range(len(ebin_centers))) - maxbin) < 15 # within 15 bins

        mpv_lo = 1
        mpv_hi = 3
        mpv0 = 2.
        p0 = [mpv0, 0.1, 0.1, np.max(N)]
        bounds = ([mpv_lo, 0.001, 0.001, np.max(N)/100.], [mpv_hi, 5, 5, np.max(N)*100.])
        popt, perr = curve_fit(landau_gaus, ebin_centers[when_fit], N[when_fit], 
                               p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N),1)[when_fit], bounds=bounds)

        plt.plot(ebin_centers, landau_gaus(ebin_centers, *popt), 
                 label="Fit")
        mpv = popt[0] - 0.228*popt[1]

        plt.text(0.45, 0.175, "10 wires\n%.0f < time < %.0f $\\mu$s\n\nMPV = %.2f MeV/cm\n$\\mu$ = %.2f MeV/cm\n\n$\eta$ = %.3f MeV/cm\n$\sigma$ = %.3f MeV/cm" %
                 tuple([blo, bhi, mpv] + list(popt[:3])),
                 fontsize=14, transform=plt.gca().transAxes)
        plt.xlabel("dE/dx [MeV/cm]")
        plt.ylabel("Entries")
        plt.title(plottitle + " TPC " + tname)

        plt.tight_layout()

        if dosave:
            plt.savefig(savedir + "10wires_t_%i_%i_dedx_TPC%s%s.png" % (int(blo), int(bhi), tname, plotqual))

        Mean_10s[tname].append(np.mean(stats.trim1(dqdx_10[thiswhen], 0.2)))
        MPV_10s[tname].append(popt[0] - 0.228*popt[1])
        MPV_err_10s[tname].append(np.sqrt(np.diag(perr)[0] + (0.228**2)*np.diag(perr)[1] - 0.228*(perr[0,1] + perr[1,0])))

        loc_10s[tname].append(popt[0])
        loc_err_10s[tname].append(np.sqrt(np.diag(perr))[0])

        eta_10s[tname].append(popt[1])
        eta_10_errs[tname].append(np.sqrt(np.diag(perr))[1])
        sigma_10s[tname].append(popt[2])
        
        ifig += 1
    
for tpc in tpcnames:
    Mean_10s[tpc] = np.array(Mean_10s[tpc])
    MPV_10s[tpc] = np.array(MPV_10s[tpc])
    MPV_err_10s[tpc] = np.array(MPV_err_10s[tpc])
    loc_10s[tpc] = np.array(loc_10s[tpc])
    loc_err_10s[tpc] = np.array(loc_err_10s[tpc])
    eta_10s[tpc] = np.array(eta_10s[tpc])
    sigma_10s[tpc] = np.array(sigma_10s[tpc])

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    
    plt.errorbar(tcenters[whent], MPV_10s[tpc][whent], MPV_err_10s[tpc][whent])
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("True MPV dE/dx [MeV/cm]")
    plt.title(plottitle + " TPC " + tpc + " 10-Wire")
    
    plt.tight_layout()

    if dosave: plt.savefig(savedir + "10wire_truededx_mpv_TPC%s%s.png" % (tpc, plotqual))

In [ ]:
tqbins = np.linspace(0, 2e5, 101)
tqbin_centers = (tqbins[1:] + tqbins[:-1]) / 2.

In [ ]:
Means = {}
MPVs = {}
MPV_errs = {}
locs = {}
loc_errs = {}

etas = {}
sigmas = {}

for tpc in tpcnames:
    Means[tpc] = []
    MPVs[tpc] = []
    MPV_errs[tpc] = []
    locs[tpc] = []
    loc_errs[tpc] = []
    etas[tpc] = []
    sigmas[tpc] = []
    
ifig = 0

for tpc, tname in zip(tpcs, tpcnames):
    for i, (blo, bhi) in enumerate(zip(tbins[:-1], tbins[1:])):
        plt.figure(ifig)
        thiswhen = tpc & when & (perhit_df.thit > blo) & (perhit_df.thit < bhi)

        N,_,_ = plt.hist(perhit_df.true_dqdx[thiswhen], bins=tqbins)

        maxbin = np.argmax(N)
        when_fit = np.abs(np.array(range(len(tqbin_centers))) - maxbin) < 15 # within 15 bins

        mpv_lo = 2e4
        mpv_hi = 1e5
        mpv0 = 5e4
        p0 = [mpv0, 1e4, 1e4, np.max(N)*1000]
        bounds = ([mpv_lo, 1e3, 1e3, np.max(N)*1000/100.], [mpv_hi, 1e5, 1e5, np.max(N)*1000*100.])
        popt, perr = curve_fit(landau_gaus, tqbin_centers[when_fit], N[when_fit], 
                               p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N),1)[when_fit], bounds=bounds)

        plt.plot(tqbin_centers, landau_gaus(tqbin_centers, *popt), 
                 label="Fit")
        mpv = popt[0] - 0.228*popt[1]

        plt.text(0.45, 0.175, "1 Wire\n%.0f < time < %.0f $\\mu$s\n\nMPV = %.0f e$^-$/cm\n$\\mu$ = %.0f e$^-$/cm\n\n$\eta$ = %.0f e$^-$/cm\n$\sigma$ = %.0f e$^-$/cm" %
                 tuple([blo, bhi, mpv] + list(popt[:3])),
                 fontsize=14, transform=plt.gca().transAxes)
        plt.xlabel("dQ/dx [e$^-$/cm]")
        plt.ylabel("Entries")
        plt.title(plottitle + " TPC " + tname)

        plt.tight_layout()

        if dosave:
            plt.savefig(savedir + "1wire_t_%i_%i_tdqdx_TPC%s%s.png" % (int(blo), int(bhi), tname, plotqual))

        Means[tname].append(np.mean(stats.trim1(perhit_df.true_dqdx[thiswhen], 0.2)))
        MPVs[tname].append(popt[0] - 0.228*popt[1])
        MPV_errs[tname].append(np.sqrt(np.diag(perr)[0] + (0.228**2)*np.diag(perr)[1] - 0.228*(perr[0,1] + perr[1,0])))

        locs[tname].append(popt[0])
        loc_errs[tname].append(np.sqrt(np.diag(perr))[0])

        etas[tname].append(popt[1])
        sigmas[tname].append(popt[2])
        
        ifig += 1
    
for tpc in tpcnames:
    Means[tpc] = np.array(Means[tpc])
    MPVs[tpc] = np.array(MPVs[tpc])
    MPV_errs[tpc] = np.array(MPV_errs[tpc])
    locs[tpc] = np.array(locs[tpc])
    loc_errs[tpc] = np.array(loc_errs[tpc])
    etas[tpc] = np.array(etas[tpc])
    sigmas[tpc] = np.array(sigmas[tpc])

In [ ]:
Mean_10s = {}
MPV_10s = {}
MPV_err_10s = {}
loc_10s = {}
loc_err_10s = {}

eta_10s = {}
sigma_10s = {}

for tpc in tpcnames:
    Mean_10s[tpc] = []
    MPV_10s[tpc] = []
    MPV_err_10s[tpc] = []
    loc_10s[tpc] = []
    loc_err_10s[tpc] = []
    eta_10s[tpc] = []
    sigma_10s[tpc] = []
    
ifig = 0

for tpc, tname in zip(tpc_10s, tpcnames):
    for i, (blo, bhi) in enumerate(zip(tbins[:-1], tbins[1:])):
        plt.figure(ifig)
        thiswhen = tpc & when10 & (thit_10 > blo) & (thit_10 < bhi)

        N,_,_ = plt.hist(true_dqdx_10[thiswhen], bins=tqbins)

        maxbin = np.argmax(N)
        when_fit = np.abs(np.array(range(len(tqbin_centers))) - maxbin) < 15 # within 15 bins

        mpv_lo = 2e4
        mpv_hi = 2e5
        mpv0 = 5e4
        p0 = [mpv0, 1e4, 1e4, np.max(N)*1000]
        bounds = ([mpv_lo, 1e3, 1e3, np.max(N)*1000/100.], [mpv_hi, 1e5, 1e5, np.max(N)*1000*100.])
        popt, perr = curve_fit(landau_gaus, tqbin_centers[when_fit], N[when_fit], 
                               p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N),1)[when_fit], bounds=bounds)

        plt.plot(tqbin_centers, landau_gaus(tqbin_centers, *popt), 
                 label="Fit")
        mpv = popt[0] - 0.228*popt[1]

        plt.text(0.45, 0.175, "10 Wire\n%.0f < time < %.0f $\\mu$s\n\nMPV = %.0f e$^-$/cm\n$\\mu$ = %.0f e$^-$/cm\n\n$\eta$ = %.0f e$^-$/cm\n$\sigma$ = %.0f e$^-$/cm" %
                 tuple([blo, bhi, mpv] + list(popt[:3])),
                 fontsize=14, transform=plt.gca().transAxes)
        plt.xlabel("dQ/dx [e$^-$/cm]")
        plt.ylabel("Entries")
        plt.title(plottitle + " TPC " + tname)

        plt.tight_layout()

        if dosave:
            plt.savefig(savedir + "10wire_t_%i_%i_tdqdx_TPC%s%s.png" % (int(blo), int(bhi), tname, plotqual))

        Mean_10s[tname].append(np.mean(stats.trim1(true_dqdx_10[thiswhen], 0.2)))
        MPV_10s[tname].append(popt[0] - 0.228*popt[1])
        MPV_err_10s[tname].append(np.sqrt(np.diag(perr)[0] + (0.228**2)*np.diag(perr)[1] - 0.228*(perr[0,1] + perr[1,0])))

        loc_10s[tname].append(popt[0])
        loc_err_10s[tname].append(np.sqrt(np.diag(perr))[0])

        eta_10s[tname].append(popt[1])
        sigma_10s[tname].append(popt[2])
        
        ifig += 1
    
for tpc in tpcnames:
    Mean_10s[tpc] = np.array(Mean_10s[tpc])
    MPV_10s[tpc] = np.array(MPV_10s[tpc])
    MPV_err_10s[tpc] = np.array(MPV_err_10s[tpc])
    loc_10s[tpc] = np.array(loc_10s[tpc])
    loc_err_10s[tpc] = np.array(loc_err_10s[tpc])
    eta_10s[tpc] = np.array(eta_10s[tpc])
    sigma_10s[tpc] = np.array(sigma_10s[tpc])

In [ ]:
p0 = [5e4, 3e3]
bounds = ([0.0, 0.0], [1e5, 10e3])

In [ ]:
for i,tpc in enumerate(tpcnames):
    plt.figure(i)
    plt.errorbar(tcenters[whent], MPVs[tpc][whent], yerr=MPV_errs[tpc][whent], label="Data")
    
    popt, perr = curve_fit(lifetime, tcenters[whent], MPVs[tpc][whent], 
                       p0=p0, maxfev=10_000, sigma=MPV_errs[tpc][whent], bounds=bounds)
    plt.plot(tcenters[whent], lifetime(tcenters[whent], *popt), label="Fit")
    plt.text(0.05, 0.05, "$\\tau = %.0f \\pm %.0f \\mu$s" % (popt[1], np.sqrt(np.diag(perr))[1]),
            fontsize=16, transform=plt.gca().transAxes)
    
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("MPV dQ/dx [e$^-$/cm]")
    plt.title(plottitle + " TPC " + tpc + " 1-Wire")
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "1wire_MPV_truedqdx_time_TPC%s%s.png" % (tpc, plotqual))

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    plt.errorbar(tcenters[whent], MPV_10s[tpc][whent], yerr=MPV_err_10s[tpc][whent])
    
    popt, perr = curve_fit(lifetime, tcenters[whent], MPV_10s[tpc][whent], 
                       p0=p0, maxfev=10_000, sigma=MPV_err_10s[tpc][whent], bounds=bounds)
    plt.plot(tcenters[whent], lifetime(tcenters[whent], *popt), label="Fit")
    plt.text(0.05, 0.05, "$\\tau = %.0f \\pm %.0f \\mu$s" % (popt[1], np.sqrt(np.diag(perr))[1]),
            fontsize=16, transform=plt.gca().transAxes)
    
    plt.title("10 Wires")
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("MPV dQ/dx [e$^-$/cm]")
    plt.title(plottitle + " TPC " + tpc + " 10-Wire")
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "10wires_MPV_truedqdx_time_TPC%s%s.png" % (tpc, plotqual))

In [ ]:
import multiprocessing
from scipy import stats

In [ ]:
def tau_stat(tau):
    Ns = []
    dqdxs = []
    for i, (blo, bhi) in enumerate(zip(tbins[:-1], tbins[1:])):
        if not ((blo+bhi) /2. > 100 and (blo+bhi)/2. < 900):
            continue                
        thiswhen = tpc & when10 & (thit_10 > blo) & (thit_10 < bhi)
        dqdx = true_dqdx_10[thiswhen]*np.exp(thit_10[thiswhen]/(tau*1e3))
        dqdx_when = dqdx < 2e5
        dqdxs.append(dqdx[dqdx_when])
        Ns.append(np.sum(dqdx_when))
    dqdxs = [np.random.choice(Q, min(Ns), False) for Q in dqdxs]
    # plt.hist(dqdxs, histtype="step", bins=tqbins)
    kstests, ps = zip(*[stats.kstest(QA, QB) for QA, QB in zip(dqdxs, dqdxs[1:] + [dqdxs[0]])])
    stat = max(kstests)
    return tau, stat

In [ ]:
taus = np.linspace(3, 4, 101)

In [ ]:
ifig = 0
for tpc, tname in zip(tpc_10s, tpcnames):
    statistics = []
    with multiprocessing.Pool(24) as pool:
        for tau, stat in pool.imap_unordered(tau_stat, taus):
            statistics.append((tau, stat))
    statistics = [s for _,s in sorted(statistics, key=lambda k: k[0])]
    plt.plot(taus, statistics)
    ifig += 1
    break

In [ ]:
print(taus[np.argmin(statistics)])

In [ ]:
def gaus(x, A, mu, sigma):
    return A*np.exp(-(x-mu)**2/sigma**2)

In [ ]:
rbins = np.linspace(0., 0.025, 26)
rbin_centers = (rbins[1:] + rbins[:-1]) / 2.

In [ ]:
ifig = 0

ratios = {}
ratio_errs = {}
for tpc in tpcnames:
    ratios[tpc] = []
    ratio_errs[tpc] = []
    
for tpc, tname in zip(tpcs, tpcnames):
    for i, (blo, bhi) in enumerate(zip(tbins[:-1], tbins[1:])):
        plt.figure(ifig)
        thiswhen = tpc & when & (perhit_df.thit > blo) & (perhit_df.thit < bhi)
        
        var = perhit_df.dqdx / perhit_df.true_dqdx
        
        N,_,_ = plt.hist(var[thiswhen], bins=rbins)
        plt.xlabel("Reco / True dQ/dx")
        plt.ylabel("Entries")
        plt.title(plottitle + " TPC " + tname + " 1-Wire")
        
        p0 = [np.max(N), 0.015, 0.001]
        bounds = ([np.max(N)/100, 0.01, 0.0], [np.max(N)*100, 0.02, 0.1])
        popt, perr = curve_fit(gaus, rbin_centers[N>0], N[N>0], 
                               p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N),1)[N>0], bounds=bounds)
        plt.plot(rbin_centers, gaus(rbin_centers, *popt))
        plt.text(0.05, 0.7, "Mean Ratio:\n%.4f $\\pm$ %.4f" % (popt[1], np.sqrt(np.diag(perr))[1]),
                fontsize="16", transform=plt.gca().transAxes)
        plt.tight_layout()
        
        ratios[tname].append(popt[1])
        ratio_errs[tname].append(np.sqrt(np.diag(perr))[1])

        ifig += 1
        
for tpc in tpcnames:
    ratios[tpc] = np.array(ratios[tpc])
    ratio_errs[tpc] = np.array(ratio_errs[tpc])

In [ ]:
ifig = 0

ratio_10s = {}
ratio_err_10s = {}
for tpc in tpcnames:
    ratio_10s[tpc] = []
    ratio_err_10s[tpc] = []
    
for tpc, tname in zip(tpc_10s, tpcnames):
    for i, (blo, bhi) in enumerate(zip(tbins[:-1], tbins[1:])):
        plt.figure(ifig)
        thiswhen = tpc & when10 & (thit_10 > blo) & (thit_10 < bhi)
        
        var = dqdx_10 / true_dqdx_10
        
        N,_,_ = plt.hist(var[thiswhen], bins=rbins)
        plt.xlabel("Reco / True dQ/dx [ADDC/e$^-$]")
        plt.ylabel("Entries")
        plt.title(plottitle + " TPC " + tname + " 10-Wire")
        
        p0 = [np.max(N), 0.015, 0.001]
        bounds = ([np.max(N)/100, 0.01, 0.0], [np.max(N)*100, 0.02, 0.1])
        popt, perr = curve_fit(gaus, rbin_centers[N>0], N[N>0], 
                               p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N),1)[N>0], bounds=bounds)
        plt.plot(rbin_centers, gaus(rbin_centers, *popt))
        plt.text(0.05, 0.7, "Mean Ratio:\n%.4f $\\pm$ %.4f" % (popt[1], np.sqrt(np.diag(perr))[1]),
                fontsize="16", transform=plt.gca().transAxes)
        plt.tight_layout()
        
        ratio_10s[tname].append(popt[1])
        ratio_err_10s[tname].append(np.sqrt(np.diag(perr))[1])

        ifig += 1

for tpc in tpcnames:
    ratio_10s[tpc] = np.array(ratio_10s[tpc])
    ratio_err_10s[tpc] = np.array(ratio_err_10s[tpc])

In [ ]:
def lin(x, m, b):
    return m*b*x + b

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    
    plt.errorbar(tcenters[whent], ratios[tpc][whent], ratio_errs[tpc][whent])
    p0 = [-1e-3,0.015 ]
    bounds = ([-1e2, 0.01], [1e2, 0.02])
    popt, perr = curve_fit(lin, tcenters[whent], ratios[tpc][whent], 
                           p0=p0, maxfev=10_000, sigma=ratio_errs[tpc][whent], bounds=bounds)
    plt.plot(tcenters[whent], lin(tcenters[whent], *popt))
    plt.text(0.01, 0.05, "Slope/Intercept: %.2f $\\pm$ %.2f %%/ms" % (popt[0]*100*1e3, np.sqrt(np.diag(perr))[0]*100*1e3),
            fontsize="16", transform=plt.gca().transAxes)
    
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("Mean Reco/True dQ/dx [ADDC/e$^-$]")
    plt.title(plottitle + " TPC " + tpc + " 1-Wire")
    plt.tight_layout()
    
    if dosave: plt.savefig(savedir + "1wire_true_reco_dqdx_ratio_tpc%s%s.png" % (tpc, plotqual))

In [ ]:
for i, tpc in enumerate(tpcnames):
    plt.figure(i)
    
    plt.errorbar(tcenters[whent], ratio_10s[tpc][whent], ratio_err_10s[tpc][whent])
    p0 = [-1e-3,0.015 ]
    bounds = ([-1e2, 0.01], [1e2, 0.02])
    popt, perr = curve_fit(lin, tcenters[whent], ratio_10s[tpc][whent], 
                           p0=p0, maxfev=10_000, sigma=ratio_err_10s[tpc][whent], bounds=bounds)
    plt.plot(tcenters[whent], lin(tcenters[whent], *popt))
    plt.text(0.01, 0.05, "Slope/Intercept: %.2f $\\pm$ %.2f %%/ms" % (popt[0]*100*1e3, np.sqrt(np.diag(perr))[0]*100*1e3),
            fontsize="16", transform=plt.gca().transAxes)
    
    plt.xlabel("Hit Time [$\\mu$s]")
    plt.ylabel("Mean Reco/True dQ/dx [ADDC/e$^-$]")
    plt.title(plottitle + " TPC " + tpc + " 10-Wire")
    plt.tight_layout()

    if dosave: plt.savefig(savedir + "10wire_true_reco_dqdx_ratio_tpc%s%s.png" % (tpc, plotqual))